In [31]:
import pandas as pd
from eunjeon import Mecab
import operator
import json
from hanspell import spell_checker
import re
from tqdm.auto import tqdm
import numpy as np
import time
import requests
from bs4 import BeautifulSoup

keywords=['나또, 낫또']
products = {}
df_box=[]
for keyword in keywords:
    r = requests.get(f'https://search.shopping.naver.com/api/search/all?fo=true&origQuery={keyword}&pagingIndex=1&pagingSize=80&productSet=total&query={keyword}').json()
    df = pd.DataFrame(r['shoppingResult']['products'])
    df_box.append(df)
df = pd.concat(df_box)
name_and_urls = df[['productTitle', 'crUrl']]

all_data = []
for name, url in tqdm(name_and_urls.values.tolist()):
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'lxml')

    for s in soup.select('script#__NEXT_DATA__'):
        script = json.loads(s.text)['props']['pageProps']['initialState']['catalog']['review']['summary']['initialReviews']
        for i in script:
            all_data.append([name, url, i['content']])

  0%|          | 0/80 [00:00<?, ?it/s]

In [33]:
mecab = Mecab()
p = re.compile('[가-힣0-9a-zA-Z]+')
spwords = ['낫', '나', '개', '때','것', '배송','주문','구매','거','수', '포장', '감사','번','저','일','중','팩','상품','가격']

word_dic={}
clean_data=[]
for name, url, review in tqdm(all_data):
    try:
        i = " ".join(p.findall(review))
        spelled_sent = spell_checker.check(i).checked
        # 리뷰 전체 데이터 오탈자 수정
        clean_data.append(spelled_sent)
        noun = mecab.nouns(spelled_sent)
        for t in noun:
            if t not in word_dic:
                word_dic[t] = 1
            else:
                word_dic[t] +=1
        try:
            for sp in spwords:
                del word_dic[sp]
        except Exception as e:
            pass
        
    except Exception as e:
        pass
    
# 총리뷰 중 최빈도 단어
fre = list(sorted(word_dic.items(), key=operator.itemgetter(1), reverse=True))       


  0%|          | 0/1038 [00:00<?, ?it/s]

In [35]:
all_df = pd.DataFrame(all_data, columns=['제품명','링크','리뷰'])
all_df['리뷰'] = clean_data
all_data = all_df.values.tolist()

output = []
for f in tqdm([ f for f,n in fre if n > 60 ]):
    for name, url, review in all_data:
        val = review.replace('em','').replace('br','').replace('  ',' ').split(' ')
        for i, s in enumerate(val):
            if f in s and len(val[i-1:i+5]) > 1:
                output.append([name, f, " ".join(val[i-1:i+5]), review, url])
pd.DataFrame(output, columns=['제품명','단어', '-1:+5gram' ,'원본 리뷰','링크']).to_excel(f'{keywords}.xlsx', index=False)

  0%|          | 0/9 [00:00<?, ?it/s]

In [37]:
output

[['풀무원 살아있는 실의 힘 국산콩 냉동 나또',
  '맛',
  '무슨 맛으로 먹나 했었는데 이게 먹다',
  '예전에 첨 접했을 땐 이걸 무슨 맛으로 먹나 했었는데 이게 먹다 보니 맛있어지더라고요 신기하게 포만감도 오래갑니다 마트에 6개나 8개들이 사면 딸애랑 둘이 먹다 보니 금방 다 먹어서 이번엔 좀 왕창 샀는데 냉동실에 들어갈 자리가 없어서 구석구석 끼워 넣었습니다 냉동상태로 잘 도착했고 배송도 빨랐으며 해동해서 먹으니 냉장유통제품과 똑같았습니다 솔직히 냉동제품이 좀 못할까 봐 은근 걱정했었거든요 앞으로는 인터넷으로 조금이라도 더 저렴하게 사서 먹어야겠어요 냉동냉장 상관없이',
  'https://cr.shopping.naver.com/adcr.nhn?x=WfSQTG0Mdm8h66kwN5vMx%2F%2F%2F%2Fw%3D%3DsGopkNCiGzszZ6o5NEcOAkOiOAPff7pmX6nqduAQoEy6qumBRc0SOCP5zLD6q%2B0A%2F0naYRyi77Vrub1tA%2FByFgc6vEgc2lSOC4D2chPX2zSUaiaH5l6%2F7wrYiVWVswUrCVhE2cL8YxJmiUg0O7DUMJEgAn8lD2%2BxrgblrBqYwPdWug%2B9sjA7rHdeMLkY%2BwIMU1gNciLEFUUeBxNnIv4nlFRiQ6zdPbhc%2FhEqgfT35LAIAxGEwVBQ%2FQ1tvkn0Igwny%2FKKmANjh3nX0PiLA1Ry7SRCyJ9Tasjyhv01Gu8Vbwlb%2Ft6%2F%2B5RIUgDbwy3Zy%2B9n0XFkBTsAEW6Se25sPVzD6tFnaNkFHrjze9qJtEkC8i3TJxazdOMMdW%2FhzVKqxe6256NdBUuOnd3BspBI2KLYMODG%2BXqS4WFHSInzUmA1eZaQulMSXmodDptJZCDmbU%2FWS8V0izaUbO337vxly3d8%2FhIv8eCH1JlmanGrOlndKJex%2Fg8y%2FMgJYDXDbObwio3u79YtwP4nG03KhnuT2iSLG4a300Na49V2